In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/22-2_인빅_프로젝트/modeling/dt_n_proc.csv', index_col=0)
train

FileNotFoundError: ignored

In [ ]:
import numpy as np
train['begin_month'] = np.abs(train['begin_month']).astype(int)

In [ ]:
train[['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'family_size', 'credit']] = train[['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'family_size', 'credit']].astype(int)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24818 entries, 0 to 26456
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         24818 non-null  int64 
 1   car            24818 non-null  int64 
 2   reality        24818 non-null  int64 
 3   income_total   24818 non-null  int64 
 4   income_type    24818 non-null  object
 5   edu_type       24818 non-null  object
 6   family_type    24818 non-null  object
 7   house_type     24818 non-null  object
 8   DAYS_BIRTH     24818 non-null  int64 
 9   DAYS_EMPLOYED  24818 non-null  int64 
 10  work_phone     24818 non-null  int64 
 11  phone          24818 non-null  int64 
 12  email          24818 non-null  int64 
 13  occyp_type     24818 non-null  object
 14  family_size    24818 non-null  int64 
 15  begin_month    24818 non-null  int64 
 16  credit         24818 non-null  int64 
dtypes: int64(12), object(5)
memory usage: 3.4+ MB


In [ ]:
train[['gender', 'car', 'reality', 'work_phone', 'phone', 'email']] =train[['gender', 'car', 'reality', 'work_phone', 'phone', 'email']].astype(object)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24818 entries, 0 to 26456
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         24818 non-null  object
 1   car            24818 non-null  object
 2   reality        24818 non-null  object
 3   income_total   24818 non-null  int64 
 4   income_type    24818 non-null  object
 5   edu_type       24818 non-null  object
 6   family_type    24818 non-null  object
 7   house_type     24818 non-null  object
 8   DAYS_BIRTH     24818 non-null  int64 
 9   DAYS_EMPLOYED  24818 non-null  int64 
 10  work_phone     24818 non-null  object
 11  phone          24818 non-null  object
 12  email          24818 non-null  object
 13  occyp_type     24818 non-null  object
 14  family_size    24818 non-null  int64 
 15  begin_month    24818 non-null  int64 
 16  credit         24818 non-null  int64 
dtypes: int64(6), object(11)
memory usage: 3.4+ MB


In [ ]:
X = train.drop('credit', axis=1)
y = train['credit']

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

clf = CatBoostClassifier()


clf.fit(X_train, y_train, 
        cat_features=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'work_phone', 'phone', 'email', 'occyp_type'], 
        eval_set=(X_val, y_val), 
        verbose=False
)

In [ ]:
clf_pred = clf.predict_proba(X_val)
from sklearn.metrics import log_loss
print(f'\tLog Loss: {log_loss(y_val, clf_pred):.6f}')

	Log Loss: 0.763127


## pool 쓴 거

In [ ]:
cat_cols = train.columns[train.dtypes == 'object'].to_list()
cat_cols

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'work_phone',
 'phone',
 'email',
 'occyp_type']

In [ ]:
params = {'iterations':200,
          'random_seed':63,
          'learning_rate':0.02,
          'loss_function':'', ## 사실 Default 값은 Logloss 이다. 만약 CatBoostRegressor 였으면, RMSE 이다.
          'custom_metric':['Logloss','AUC'],##, '
          'early_stopping_rounds':20,
          'use_best_model': True,
          'bagging_temperature':1,
          'verbose':False}

In [ ]:
from catboost import Pool
train_pool = Pool(X_train, y_train, cat_features = cat_cols)
eval_pool = Pool(X_val, y_val, cat_features = cat_cols)

In [ ]:
model = CatBoostClassifier() ##괄호 안에 params 잘 넣어서 조절해보기
model.fit(train_pool, eval_set=eval_pool,early_stopping_rounds=100, verbose=100) ## ,save_snapshot=True

Learning rate set to 0.114626
0:	learn: 1.0377794	test: 1.0356834	best: 1.0356834 (0)	total: 89ms	remaining: 1m 28s
100:	learn: 0.8018915	test: 0.7921486	best: 0.7921486 (100)	total: 6.56s	remaining: 58.4s
200:	learn: 0.7696370	test: 0.7823881	best: 0.7823881 (200)	total: 14.3s	remaining: 56.8s
300:	learn: 0.7442417	test: 0.7775697	best: 0.7775697 (300)	total: 21.9s	remaining: 50.9s
400:	learn: 0.7216778	test: 0.7742115	best: 0.7740587 (391)	total: 29.7s	remaining: 44.4s
500:	learn: 0.6984548	test: 0.7708685	best: 0.7708685 (500)	total: 37.7s	remaining: 37.5s
600:	learn: 0.6765443	test: 0.7686250	best: 0.7686211 (598)	total: 46.3s	remaining: 30.7s
700:	learn: 0.6571215	test: 0.7663195	best: 0.7661170 (690)	total: 54.2s	remaining: 23.1s
800:	learn: 0.6386205	test: 0.7644772	best: 0.7644343 (799)	total: 1m 3s	remaining: 15.8s
900:	learn: 0.6206432	test: 0.7635415	best: 0.7631268 (842)	total: 1m 12s	remaining: 7.92s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7631

In [ ]:
pred_train = model.predict_proba(X_train)
pred_val = model.predict_proba(X_val)

In [ ]:
print(f'\tLog Loss: {log_loss(y_val, pred_val):.6f}')

	Log Loss: 0.763127


In [ ]:
import pandas as pd

In [ ]:
pred = pd.DataFrame(pred_train)

In [ ]:
pred

,0,1,2
0,0.076714,0.268260,0.655026
1,0.099708,0.156646,0.743646
2,0.090321,0.102659,0.807020
3,0.169023,0.201121,0.629856
4,0.084983,0.270579,0.644438
...,...,...,...
19849,0.203239,0.129346,0.667415
19850,0.093378,0.086492,0.820131
19851,0.063788,0.929550,0.006661
19852,0.088571,0.211420,0.700009


In [ ]:
from sklearn.metrics import accuracy_score
print('{:.4}'.format(accuracy_score(y_val, model.predict(X_val))))

0.7031


여기서 부터는 확률을 올리기 위해 col drop 할 것임.

In [3]:
pd.DataFrame({'feature_importance': model.get_feature_importance(train_pool), 
              'feature_names': X_val.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

NameError: ignored